In [ ]:
#Read Colab secrets and export to env vars expected by the notebook
# Read Colab secrets and export to env vars expected by the notebook
import os
try:
    from google.colab import userdata
except Exception:
    userdata = None

def _get(*names):
    for n in names:
        try:
            v = userdata.get(n) if userdata else None
        except Exception:
            v = None
        if v:
            return v.strip()
    return None

groq = _get("GROQ_API_KEY", "GROQ_API")
gemini = _get("GEMINI_API_KEY", "GEMINI_API", "GOOGLE_API_KEY")

if groq:  os.environ["GROQ_API_KEY"] = groq
if gemini: os.environ["GEMINI_API_KEY"] = gemini

print("GROQ set:", bool(os.environ.get("GROQ_API_KEY")))
print("GEMINI set:", bool(os.environ.get("GEMINI_API_KEY")))


GROQ set: True
GEMINI set: True


In [ ]:
import os
print(os.environ.get("GROQ_API_KEY", "")[:6] + "…")
print(os.environ.get("GEMINI_API_KEY", "")[:6] + "…")

gsk_VS…
AIzaSy…


In [ ]:
# @title Install + Imports
!pip -q install gradio groq google-generativeai pandas python-dateutil dateparser

import os, re, json, tempfile
import gradio as gr
import pandas as pd
from datetime import datetime
import dateparser

USE_GROQ = bool(os.environ.get("GROQ_API_KEY"))
USE_GEMINI = bool(os.environ.get("GEMINI_API_KEY"))

if not (USE_GROQ or USE_GEMINI):
    print("🔑 Set at least one key before running: "
          "os.environ['GROQ_API_KEY']='...' or os.environ['GEMINI_API_KEY']='...'")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 18.6 MB/s eta 0:00:00


In [ ]:
# @title LLM helper
def llm_call(system: str, user: str, temperature: float = 0.2) -> str:
    if USE_GROQ:
        from groq import Groq
        client = Groq(api_key=os.environ["GROQ_API_KEY"])
        resp = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[{"role": "system", "content": system},
                      {"role": "user", "content": user}],
            temperature=temperature,
        )
        print("resp:", resp)
        return resp.choices[0].message.content.strip()

    if USE_GEMINI:
        import google.generativeai as genai
        genai.configure(api_key=os.environ["GEMINI_API_KEY"])
        model = genai.GenerativeModel("gemini-1.5-flash")
        prompt = f"System:\n{system}\n\nUser:\n{user}"
        resp = model.generate_content(prompt)
        return resp.text.strip()

    raise RuntimeError("No API key found. Set GROQ_API_KEY or GEMINI_API_KEY.")

In [ ]:
# @title Agent logic
SYSTEM_PROMPT = """You are a precise Meeting Notes Assistant.
Given raw notes, return a compact JSON with:
- summary: exactly 3 bullet points (short phrases).
- decisions: list of decisions (0..5 concise items).
- action_items: list of objects with keys:
  task (imperative), owner (first name or role), due_date (natural language OK),
  priority (High|Medium|Low). Only include real, actionable tasks.
- email_subject: short subject line for a follow-up email.
- email_body: 120-180 words, crisp recap + action items with owners & due dates.
STRICTLY return only a JSON object. No code fences, no commentary.
Keep writing professional, friendly, and unambiguous.
"""

USER_TEMPLATE = """MEETING:
Title: {title}
DateTime (local): {meeting_dt}
Notes:
{notes}

Constraints:
- summary: exactly 3 bullets.
- If a due date is hinted (e.g., “next Friday”), include it; else use "TBD".
- Prioritize tasks that move metrics or remove blockers.
Return only the JSON object.
"""

ALLOWED_PRIORITIES = {"High","Medium","Low"}

def parse_json_loose(text: str) -> dict:
    m = re.search(r"\{.*\}", text, flags=re.S)
    candidate = m.group(0) if m else text
    return json.loads(candidate)

def normalize_due_date(due_str: str, base_dt: datetime) -> str:
    if not due_str or due_str.strip().upper() == "TBD":
        return "TBD"
    dt = dateparser.parse(
        due_str,
        settings={"RELATIVE_BASE": base_dt, "PREFER_DATES_FROM": "future"}
    )
    return dt.date().isoformat() if dt else due_str

In [ ]:
# @title Gradio UI (runs in Colab)
def run_agent(title, meeting_dt, notes):
    # Fallbacks
    title = (title or "Team Sync").strip()
    meeting_dt = (meeting_dt or datetime.now().strftime("%Y-%m-%d %H:%M")).strip()
    notes = (notes or "").strip()

    user = USER_TEMPLATE.format(title=title, meeting_dt=meeting_dt, notes=notes)
    raw = llm_call(SYSTEM_PROMPT, user)

    try:
        js = parse_json_loose(raw)
    except Exception as e:
        return ("", f"⚠️ Could not parse model output:\n{raw}\n\nError: {e}",
                pd.DataFrame(), None)

    # Normalize action items
    try:
        base_dt = datetime.fromisoformat(meeting_dt.replace("Z","").strip())
    except:
        base_dt = datetime.now()

    for item in js.get("action_items", []):
        pr = str(item.get("priority","")).title()
        item["priority"] = pr if pr in ALLOWED_PRIORITIES else "Medium"
        item["due_date"] = normalize_due_date(item.get("due_date",""), base_dt)

    # Build outputs
    summary_md = "### Summary (3 bullets)\n" + "\n".join([f"- {b}" for b in js.get("summary", [])])
    decisions = js.get("decisions", [])
    decisions_md = "### Decisions\n" + ("\n".join([f"- {d}" for d in decisions]) if decisions else "- (none)")

    # Dataframe + CSV
    rows = []
    for i, ai in enumerate(js.get("action_items", []), 1):
        rows.append({
            "idx": i,
            "task": ai.get("task",""),
            "owner": ai.get("owner",""),
            "due_date": ai.get("due_date",""),
            "priority": ai.get("priority",""),
        })
    ai_df = pd.DataFrame(rows) if rows else pd.DataFrame(columns=["idx","task","owner","due_date","priority"])

    csv_path = None
    if len(ai_df) > 0:
        tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
        ai_df.to_csv(tmp.name, index=False)
        csv_path = tmp.name

    email_block = f"*Subject:* {js.get('email_subject','')}\n\n{js.get('email_body','')}"
    return summary_md + "\n\n" + decisions_md, email_block, ai_df, csv_path

with gr.Blocks() as demo:
    gr.Markdown("## Meeting Minutes → Action-Items Agent (Colab-friendly, Groq/Gemini)")
    with gr.Row():
        title_in = gr.Textbox(label="Meeting Title", value="Q3 Growth Marketing Sync")
        dt_in = gr.Textbox(label="Meeting Date/Time (local)", value=datetime.now().strftime("%Y-%m-%d %H:%M"))
    notes_in = gr.Textbox(label="Raw Notes", lines=12, placeholder="Paste raw notes here...")

    run_btn = gr.Button("Extract Summary, Decisions, Action Items")

    out_summary = gr.Markdown()
    out_email   = gr.Markdown()
    out_table   = gr.Dataframe(headers=["idx","task","owner","due_date","priority"], interactive=False)
    out_file    = gr.File(label="Download Action Items CSV")

    run_btn.click(run_agent, inputs=[title_in, dt_in, notes_in],
                  outputs=[out_summary, out_email, out_table, out_file])

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fb7d8343efc779e72c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
